# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [53]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [54]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

# write to csv file 
# city_data_df.to_csv("../output_data/cities.csv", index=False)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,239.25,99,100,0.89,RU,1707523721
1,1,adamstown,-25.0660,-130.1015,298.44,90,100,10.85,PN,1707523722
2,2,wailua homesteads,22.0669,-159.3780,298.59,69,40,4.63,US,1707523722
3,3,port mathurin,-19.6833,63.4167,299.88,79,94,11.91,MU,1707523722
4,4,iqaluit,63.7506,-68.5145,241.09,86,13,3.54,CA,1707523723


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [55]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size = 'Humidity',
    scale=0.5,
    c='City', #sets color based on city name 
    ylim=(85, -70),
    geo=True, 
    tiles='OSM',
    title='Cities and Their Relative Humidity'
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [56]:
# pull a city for reference 
benchmark= city_data_df.loc[city_data_df['City'] == 'guerrero negro']
benchmark

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,guerrero negro,27.9769,-114.0611,288.6,57,58,8.4,MX,1707523461


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values 
# max temp 240 kelvin, wind speed < 15, cloudiness < 75, would prefer a democratic country 

# Define democratic countries 
democratic_countries = [
    'US', # United States
    'CA', # Canada
    'AU', # Australia
    'NZ', # New Zealand
    'ZA', # South Africa
    'AR', # Argentina
    'ES', # Spain
    'DE', # Germany
    'JP', # Japan
    'FR', # France
    'GB', # United Kingdom
    'IN', # India
    'BR', # Brazil
    'MX', # Mexico
    'KR', # South Korea
    'IT', # Italy
    'PT', # Portugal
    'SE', # Sweden
    'NO', # Norway
    'DK', # Denmark
    'FI', # Finland
    'IS', # Iceland
    'IE', # Ireland
    'NL', # Netherlands
    'BE', # Belgium
    'LU', # Luxembourg
    'AT', # Austria
    'CH', # Switzerland
    'TW', # Taiwan
    'PL', # Poland
    'CZ', # Czech Republic
    'SK', # Slovakia
    'SI', # Slovenia
    'EE', # Estonia
    'LV', # Latvia
    'LT', # Lithuania
    'HR', # Croatia
    'GR', # Greece
    'HU', # Hungary
    'CY', # Cyprus
    'MT', # Malta
]

# Filter the DataFrame to find the cities that fit the criteria
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 280) & 
    (city_data_df['Wind Speed'] < 20) & 
    (city_data_df['Cloudiness'] < 50) &
    (city_data_df['Country'].isin(democratic_countries))
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(50)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,iqaluit,63.7506,-68.5145,241.09,86,13,3.54,CA,1707523723
34,34,dawson creek,55.7666,-120.2362,268.28,82,44,1.28,CA,1707523728
35,35,grand falls-windsor,48.9332,-55.6649,260.98,100,48,1.29,CA,1707523729
73,73,yellowknife,62.4560,-114.3525,255.44,78,40,3.60,CA,1707523445
85,85,bomdila,27.2500,92.4000,276.06,65,1,2.75,IN,1707523738
158,158,glace bay,46.1969,-59.9570,269.14,93,8,0.89,CA,1707523755
168,168,keflavik,64.0049,-22.5624,268.39,74,0,5.66,IS,1707523757
171,171,malvik,63.4333,10.6833,262.87,79,4,5.14,NO,1707523758
308,308,jogeva,58.7467,26.3939,256.42,97,8,1.12,EE,1707523840
358,358,namie,37.4833,141.0000,277.91,72,10,1.56,JP,1707523855


In [58]:
###TEST###
clean_city_data_df = city_data_df.dropna(subset=['Humidity'])
clean_city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,239.25,99,100,0.89,RU,1707523721
1,1,adamstown,-25.0660,-130.1015,298.44,90,100,10.85,PN,1707523722
2,2,wailua homesteads,22.0669,-159.3780,298.59,69,40,4.63,US,1707523722
3,3,port mathurin,-19.6833,63.4167,299.88,79,94,11.91,MU,1707523722
4,4,iqaluit,63.7506,-68.5145,241.09,86,13,3.54,CA,1707523723
...,...,...,...,...,...,...,...,...,...,...
535,535,bamboo flat,11.7000,92.7167,294.20,100,20,0.51,IN,1707523897
536,536,tokar,18.4254,37.7290,296.26,78,7,3.28,SD,1707523897
537,537,northam,51.0333,-4.2167,282.40,97,88,4.36,GB,1707523897
538,538,gumushacikoy,40.8731,35.2147,273.33,76,6,1.92,TR,1707523898


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
lat = hotel_df['Lat']
lng = hotel_df['Lng']

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,iqaluit,63.7506,-68.5145,241.09,86,13,3.54,CA,1707523723,
34,34,dawson creek,55.7666,-120.2362,268.28,82,44,1.28,CA,1707523728,
35,35,grand falls-windsor,48.9332,-55.6649,260.98,100,48,1.29,CA,1707523729,
73,73,yellowknife,62.4560,-114.3525,255.44,78,40,3.60,CA,1707523445,
85,85,bomdila,27.2500,92.4000,276.06,65,1,2.75,IN,1707523738,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lon},{Lat},{radius}"
    params["bias"] = f"proximity:{Lon},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
dawson creek - nearest hotel: Comfort Inn
grand falls-windsor - nearest hotel: Hotel Robin Hood
yellowknife - nearest hotel: No hotel found
bomdila - nearest hotel: Seagull
glace bay - nearest hotel: No hotel found
keflavik - nearest hotel: Núpan Deluxe
malvik - nearest hotel: Vikhammer Motell
jogeva - nearest hotel: Coop Hotell Jõgeva
namie - nearest hotel: HOTELなみえ
hammerfest - nearest hotel: Thon Hotel Hammerfest
bristol - nearest hotel: No hotel found
corner brook - nearest hotel: No hotel found
seosan - nearest hotel: 호텔씨에프


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,iqaluit,63.7506,-68.5145,241.09,86,13,3.54,CA,1707523723,Frobisher Inn
34,34,dawson creek,55.7666,-120.2362,268.28,82,44,1.28,CA,1707523728,Comfort Inn
35,35,grand falls-windsor,48.9332,-55.6649,260.98,100,48,1.29,CA,1707523729,Hotel Robin Hood
73,73,yellowknife,62.4560,-114.3525,255.44,78,40,3.60,CA,1707523445,No hotel found
85,85,bomdila,27.2500,92.4000,276.06,65,1,2.75,IN,1707523738,Seagull
158,158,glace bay,46.1969,-59.9570,269.14,93,8,0.89,CA,1707523755,No hotel found
168,168,keflavik,64.0049,-22.5624,268.39,74,0,5.66,IS,1707523757,Núpan Deluxe
171,171,malvik,63.4333,10.6833,262.87,79,4,5.14,NO,1707523758,Vikhammer Motell
308,308,jogeva,58.7467,26.3939,256.42,97,8,1.12,EE,1707523840,Coop Hotell Jõgeva
358,358,namie,37.4833,141.0000,277.91,72,10,1.56,JP,1707523855,HOTELなみえ


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [69]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    size = 'Humidity',
    scale=0.5,
    c='City',  
    ylim=(85, -70),
    hover_cols=['Hotel Name'],
    geo=True, 
    tiles='OSM',
    title='Cities and Their Relative Humidity'
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)